In [ ]:
from numpy import load
from os.path import join

def count_split(model_dir: str):
    split_count = (
        len(load(join(model_dir, "split", "train.npy"))),
        len(load(join(model_dir, "split", "val.npy"))),
        len(load(join(model_dir, "split", "test.npy")))
    )
    print(f"train: {split_count[0]}, test: {split_count[1]}, val: {split_count[2]}")


In [ ]:
model_path = "data/runs/hpe_dnn/arch1-kf-fold1"

count_split(model_path)

In [ ]:
model_path = "data/runs/hpe_dnn/arch1-full-kf-fold1"

count_split(model_path)

In [ ]:
from src.hpe.common.landmarks import YoloLabels, MyLandmark

def count_landmarks(labels: YoloLabels) -> int:
    
    def count_landmark(landmark: MyLandmark) -> int:
        label = labels.get_keypoint(landmark)
        return 0 if label.is_missing() else 1
    
    return sum(list(map(count_landmark, MyLandmark)))

In [ ]:
from os import listdir
from os.path import join

from src.hpe.common.landmarks import build_yolo_labels, get_most_central

root_label_dir = "data/hpe/img/test/labels"
totals = 0
    
for label_name in listdir(root_label_dir):
    label_path = join(root_label_dir, label_name)
    df = build_yolo_labels(label_path)
    
    if len(df) == 0:
        continue
    elif len(df) == 1:
        totals += count_landmarks(df[0])
    else:
        totals += count_landmarks(get_most_central(df))



In [ ]:
totals / 41

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-unbalanced-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-mp-dr0.3-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-yolo-kf"),
    # HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    # HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))